In [1]:
import os
import sys
import math
import pandas as pd

In [2]:
root_dir = '/home/SENSETIME/fengxiaotong/Desktop/code/common_math'
athena_dir = os.path.join(root_dir, 'athena')
data_raw_path = os.path.join(athena_dir,'data_demo','raw')
sub_dirs = [d for d in os.listdir(data_raw_path) if os.path.isdir(os.path.join(data_raw_path, d))]
sub_dirs

['000a07a3-fec1-47d5-8ba7-e8610409efb3']

In [3]:
agent_parquet_path = os.path.join(data_raw_path,sub_dirs[0],f'scenario_{sub_dirs[0]}.parquet')
df = pd.read_parquet(agent_parquet_path)
# df
data_sense_raw_path = os.path.join(athena_dir,'data_demo','raw_sense')
sub_dirs = [d for d in os.listdir(data_sense_raw_path) if os.path.isdir(os.path.join(data_sense_raw_path, d))]
agent_parquet_path = os.path.join(data_sense_raw_path,sub_dirs[0],f'scenario_{sub_dirs[0]}.parquet')

df_sense = pd.read_parquet(agent_parquet_path)
# df_sense

In [4]:
cmake_dir = os.path.join(athena_dir,'cmake-build-debug')
sys.path.append(cmake_dir)
import pybind_dlp

In [8]:
param = pybind_dlp.AgentDataParam()
param.history_num = 50
param.predict_num = 60
param.mode = pybind_dlp.AgentInfoMode.Train

manager = pybind_dlp.AgentInfoManager(param)
map_data_param = pybind_dlp.MapDataParam()
map_data_param.mode = pybind_dlp.MapDataMode.M_TRAIN
map_data_param.max_polygon_num = 30
map_data_param.max_point_num = 500

map_info_manager = pybind_dlp.MapInfoManager(map_data_param)
count = 0
frame_info = pybind_dlp.AgentFrameInfo()
for time_step in range(param.history_num + param.predict_num):
    df_slice = df[df['timestep'] == time_step]
    for index,row in df_slice.iterrows():
        if row['track_id'] == 'AV':
            count = count + 1
        if 'object_category' in row:
#             argoverse data
            if not row['track_id'] == 'AV' and (not (row['object_category'] == 2 or row['object_category'] == 3)):
                continue
        frame_info.rel_time_step = 0
        frame_info.position_x = row['position_x']
        frame_info.position_y = row['position_y']
        frame_info.position_z = 0.0
        frame_info.heading = row['heading']
        frame_info.velocity = math.sqrt(row['velocity_x']**2 + row['velocity_y']**2)
        frame_info.acceleration = 0.0
        if row['track_id'] == 'AV':
            frame_info.id = -1
        else:
            frame_info.id = int(row['track_id'])
        if row['object_type'] == 'vehicle':
            frame_info.type = pybind_dlp.AgentType.VEHICLE
        elif row['object_type'] == 'pedestrian':
            frame_info.type = pybind_dlp.AgentType.PEDESTRIAN
        elif row['object_type'] == 'motorcyclist':
            frame_info.type = pybind_dlp.AgentType.MOTORCYCLIST
        elif row['object_type'] == 'cyclist':
            frame_info.type = pybind_dlp.AgentType.CYCLIST
        elif row['object_type'] == 'bus':
            frame_info.type = pybind_dlp.AgentType.BUS
        else:
            frame_info.type = pybind_dlp.AgentType.OTHERS
        if row['track_id'] == 'AV':
            if 'intersection_flag' in row:
#                 senseauto date
                if row['turn_light'] == 'left':
                    frame_info.type = pybind_dlp.AgentType.AV_LEFT
                elif row['turn_light'] == 'left':
                    frame_info.type = pybind_dlp.AgentType.AV_RIGHT
                else:
                    frame_info.type = pybind_dlp.AgentType.AV_STRAIGHT
            else:
#                 argoverse date
                frame_info.type = pybind_dlp.AgentType.AV_STRAIGHT
        frame_info.is_static = frame_info.velocity < 0.1
        manager.push_agent(frame_info)
    manager.update(map_info_manager)
    # count = count + 1
manager.data_preprocess()
# print(count)
agent_map = manager.get_agent_info_map()

 id 5565pos_x 539.607pos_y 3930.25heading 1.28806 lon 1.21553 lat 0.159544
 id 5694pos_x 537.05pos_y 3921.84heading 1.3516 lon 0.667946 lat -0.00778611
 id 6049pos_x 536.351pos_y 3904.33heading 1.36514 lon -0.410274 lat 0.773889
 id 6452pos_x 542.291pos_y 3927.33heading 1.25738 lon 1.07382 lat 0.973109
 id 6053pos_x 533.577pos_y 3892.92heading 1.36275 lon -1.14349 lat 0.716384
 id 6167pos_x 532.455pos_y 3902.16heading 1.3533 lon -0.595509 lat -0.0590321
 id 6294pos_x 537.591pos_y 3910.86heading 1.35161 lon 0.0053764 lat 0.721112
 id -1pos_x 534.757pos_y 3911.41heading 1.35167 lon 0 lat 0
 id 6329pos_x 540.054pos_y 3920.02heading 1.29416 lon 0.597757 lat 0.824156
 id 6728pos_x 533.678pos_y 3898.93heading 1.28671 lon -0.775498 lat 0.414573
 id 6926pos_x 539.945pos_y 3952.12heading -1.67807 lon 2.55434 lat -0.946606


In [9]:
print(agent_map.keys())
len(agent_map[-1].get_historical_info())
print(manager.mask_id)

dict_keys([5565, 5694, 6049, 6452, 6053, 6167, 6294, -1, 6329, 6728, 6926])
[-1, 6167, 5694, 6294, 6049, 6728, 6329, 5565, 6053, 6452, 6926]


In [10]:
# def get_case_feature(map_path,agent_data) -> dict:
#     try:
#         with open(map_path, 'r') as f:
#             map_data = json.load(f)
#     except FileNotFoundError:
#         print("文件未找到")
    
#     av_result = agent_data[agent_data['track_id'] == 'AV']

#     av_result_filtered = av_result[(av_result['timestep'] >= 50) & (av_result['timestep'] <= 55)]

#     near_dis = 8
#     in_intersect = True
#     traffic_light = 'neutral'
#     for index, row in av_result_filtered.iterrows():
#         if 'turn_light' in av_result_filtered.columns and row['turn_light'] != 'neutral':
#             traffic_light = row['turn_light']
#         near_centerline = False
#         for lane_seg in map_data['lane_segments']:
#             if map_data['lane_segments'][lane_seg]['is_intersection']:
#                 continue
#             for pt in map_data['lane_segments'][lane_seg]['centerline']:
#                 if math.sqrt((pt['x'] - row['position_x']) ** 2 + (pt['y'] - row['position_y']) **2) < near_dis:
#                     near_centerline = True
#                     break
#             if near_centerline:
#                 break
#         if near_centerline:
#             in_intersect = False
#             break

#     return {
#         'in_intersect': in_intersect,
#         'traffic_light': traffic_light
#     }

In [11]:
pos_all = []
ego_all = []

predict_pos = []
predict_id = []

target_time = -60

for id_ in agent_map.keys():
    pos_x = []
    pos_y = []
    for his_info in agent_map[id_].get_historical_info():
        pos_x.append(his_info.position_x)
        pos_y.append(his_info.position_y)
        if his_info.rel_time_step == target_time:
            predict_id.append(id_)
            predict_pos.append(
            {
                'x':his_info.position_x,
                'y':his_info.position_y,
                'theta':his_info.heading
            }
            )
    print(id_)
    if id_ > 0:
        pos_all.append(
        {
            'x': pos_x,
            'y': pos_y
        })
    else:
        ego_all.append(
        {
            'x': pos_x,
            'y': pos_y
        })


5565
5694
6049
6452
6053
6167
6294
-1
6329
6728
6926


In [12]:
x_coords = [point['x'] for point in predict_pos]
y_coords = [point['y'] for point in predict_pos]
predict_id,predict_pos

([5565, 5694, 6049, 6452, 6053, 6167, 6294, -1, 6329, 6728, 6926],
 [{'x': 539.6074425364972,
   'y': 3930.250276107717,
   'theta': 1.2880593923180652},
  {'x': 537.0495708550417,
   'y': 3921.843941838492,
   'theta': 1.3516033329336703},
  {'x': 536.3513370920473,
   'y': 3904.3252750240085,
   'theta': 1.365144107109235},
  {'x': 542.2910140126422,
   'y': 3927.329796525354,
   'theta': 1.2573822385657065},
  {'x': 533.5766106329135,
   'y': 3892.9243320677765,
   'theta': 1.3627517055775997},
  {'x': 532.4550604284276, 'y': 3902.156642671481, 'theta': 1.353296854047129},
  {'x': 537.5909536685949,
   'y': 3910.8625401412573,
   'theta': 1.3516114637111312},
  {'x': 534.756783849463,
   'y': 3911.4056004454287,
   'theta': 1.3516652128663875},
  {'x': 540.0536362507459,
   'y': 3920.024369545462,
   'theta': 1.2941633717236165},
  {'x': 533.6781556454484, 'y': 3898.933860860753, 'theta': 1.286709627224507},
  {'x': 539.9451769439545,
   'y': 3952.1208563836926,
   'theta': -1.67807

In [13]:
from bokeh.plotting import figure, output_notebook, show
from bokeh.models import ColumnDataSource, LabelSet
output_notebook()

Loading BokehJS ...

In [14]:

p = figure(title="Agent Position", x_axis_label='X Position', y_axis_label='Y Position',match_aspect=True)

for item in pos_all:
    p.scatter(item['x'], item['y'], size=10, color="navy", alpha=0.5)
for item in ego_all:
    p.scatter(item['x'], item['y'], size=10, color="red", alpha=0.5)

source = ColumnDataSource(data=dict(x=x_coords, y=y_coords, id=predict_id))

p.scatter('x', 'y', size=10, source=source)

labels = LabelSet(x='x', y='y', text='id', level='glyph',
                  x_offset=5, y_offset=5, source=source)

p.add_layout(labels)
show(p)

In [15]:
print(manager.mask_id)

[-1, 6167, 5694, 6294, 6049, 6728, 6329, 5565, 6053, 6452, 6926]


In [16]:
mask_vaild = manager.agent_output_tensor().get_agent_valid_mask()
mask_valid_data = mask_vaild.data()
mask_valid_data[0]

[False,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True]

In [17]:
import numpy as np
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.transform import linear_cmap
from bokeh.models import ColorBar, LinearColorMapper
from bokeh.layouts import layout

In [18]:
mask_numeric = np.array(mask_valid_data, dtype=int)

mapper = LinearColorMapper(palette=["#ff7f0e", "#1f77b4"], low=0, high=1)

p = figure(title="Agent Mask Valid",
           x_range=(0, mask_numeric.shape[1]), 
           y_range=(0, mask_numeric.shape[0]), 
           toolbar_location=None)

p.image(image=[mask_numeric], 
        x=0, 
        y=0, 
        dw=mask_numeric.shape[1], 
        dh=mask_numeric.shape[0], 
        color_mapper=mapper)

# color_bar = ColorBar(color_mapper=mapper, location=(0, 0))
# p.add_layout(color_bar, 'right')

p.grid.grid_line_color = None
p.axis.axis_line_color = None
p.axis.major_tick_line_color = None
p.axis.major_label_standoff = 0
p.axis.major_label_text_font_size = "0pt"

show(p)


In [19]:
mask_predict = manager.agent_output_tensor().get_agent_predict_mask()
mask_predict = mask_predict.data()

In [20]:
mask_numeric = np.array(mask_predict, dtype=int)

mapper = LinearColorMapper(palette=["#ff7f0e", "#1f77b4"], low=0, high=1)

p = figure(title="Agent Mask Valid",
           x_range=(0, mask_numeric.shape[1]), 
           y_range=(0, mask_numeric.shape[0]), 
           toolbar_location=None)

p.image(image=[mask_numeric], 
        x=0, 
        y=0, 
        dw=mask_numeric.shape[1], 
        dh=mask_numeric.shape[0], 
        color_mapper=mapper)

# color_bar = ColorBar(color_mapper=mapper, location=(0, 0))
# p.add_layout(color_bar, 'right')

p.grid.grid_line_color = None
p.axis.axis_line_color = None
p.axis.major_tick_line_color = None
p.axis.major_label_standoff = 0
p.axis.major_label_text_font_size = "0pt"

show(p)

In [21]:
type_vaild = manager.agent_output_tensor().get_agent_type()
type_vaild.data()

[5, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 8, 8, 8, 8]

In [22]:
agent_position = manager.agent_output_tensor().get_agent_position()
agent_position_data = agent_position.data()

In [23]:
from bokeh.layouts import gridplot

In [24]:
# 提取所有 agent 的 x, y 坐标
x_coords = []
y_coords = []

for i in range(len(agent_position_data)):
    for j in range(len(agent_position_data[i])):
        if mask_valid_data[i][j]:
            x_coords.append(agent_position_data[i][j][0])  # x坐标
            y_coords.append(agent_position_data[i][j][1])  # y坐标

# 创建一个图形对象
p = figure(title="Agent Positions", x_axis_label='X', y_axis_label='Y', match_aspect=True)

# 在图上绘制所有点
p.scatter(x_coords, y_coords, size=8, color="navy", alpha=0.5)

# 显示图形
show(p)

In [25]:
agent_heading = manager.agent_output_tensor().get_agent_heading()
agent_heading_data = agent_heading.data()

In [26]:
from bokeh.plotting import figure, show
from bokeh.models import Arrow, VeeHead
import math

# 提取所有 agent 的 x, y 坐标以及 heading 信息
x_coords = []
y_coords = []
headings = []

for i in range(len(agent_position_data)):
    for j in range(len(agent_position_data[i])):
        if mask_valid_data[i][j]:
            x_coords.append(agent_position_data[i][j][0])  # x 坐标
            y_coords.append(agent_position_data[i][j][1])  # y 坐标
            headings.append(agent_heading_data[i][j])  # heading 值

# 创建一个图形对象
p = figure(title="Agent Positions with Headings", x_axis_label='X', y_axis_label='Y', match_aspect=True)

# 在图上绘制所有点
p.scatter(x_coords, y_coords, size=8, color="red", alpha=0.5)

# 为每个点添加朝向箭头
arrow_length = 0.5  # 箭头的长度，可以根据需要调整

for i in range(len(x_coords)):
    x_start = x_coords[i]
    y_start = y_coords[i]
    heading = headings[i]

    # 计算箭头的终点坐标
    x_end = x_start + arrow_length * math.cos(heading)
    y_end = y_start + arrow_length * math.sin(heading)

    # 添加箭头到图形对象
    arrow = Arrow(end=VeeHead(size=10), x_start=x_start, y_start=y_start, 
                  x_end=x_end, y_end=y_end, line_width=2, line_color="red")
    p.add_layout(arrow)

# 显示图形
show(p)

In [27]:
agent_v = manager.agent_output_tensor().get_agent_velocity()
agent_v_data = agent_v.data()

In [28]:
# 提取所有 agent 的 x, y 坐标以及 heading 信息
x_coords = []
y_coords = []
headings = []

for i in range(len(agent_v_data)):
    for j in range(len(agent_v_data[i])):
        if mask_valid_data[i][j]:
            x_coords.append(agent_v_data[i][j][0])  # x 坐标
            y_coords.append(agent_v_data[i][j][1])  # y 坐标
           
# 创建一个图形对象
p = figure(title="Agent v ", x_axis_label='X', y_axis_label='Y', match_aspect=True)

# 在图上绘制所有点
p.scatter(x_coords, y_coords, size=8, color="red", alpha=0.5)

# 显示图形
show(p)

In [29]:
agent_x_a = manager.agent_output_tensor().get_agent_enc_x_a()
agent_x_a_data_raw = agent_x_a.data()
agent_x_a_data = []
count = 0
for item in agent_x_a_data_raw:
    count = count + 1
    if count < 5:
        continue
    agent_x_a_data.append(item)
    break

In [30]:
# 假设 agent_x_a_data 是一个 N*T*4 的三维张量
# 初始化图形列表
plots = []

# 循环遍历第四维度 (index from 0 to 3)
for i in range(4):
    x_all = []
    y_all = []
    # 选择一个特定的维度进行绘图
    for n in range(len(agent_x_a_data)):
        x_coords = []
        y_coords = []
        for t in range(len(agent_x_a_data[n])):
            x_coords.append(t)  # 时间轴 t 作为 x 轴
            y_coords.append(agent_x_a_data[n][t][i])  # 第四维度的值作为 y 轴
        x_all.append(x_coords)
        y_all.append(y_coords)

    # 创建一个图形对象
    p = figure(title=f"Dimension {i+1}", x_axis_label='Time (T)', y_axis_label=f'Dimension {i+1} Values')
    
    # 在图上绘制曲线
    p.multi_line(x_all, y_all, line_width=2)

    # 保存该图形到列表中
    plots.append(p)

# 创建一个 2x2 的布局，将所有图形显示在一个网格中
grid = gridplot([plots[:2], plots[2:]])

# 显示图形
show(grid)

In [31]:
from bokeh.plotting import figure, show
from bokeh.layouts import gridplot
from bokeh.models import WheelZoomTool, PanTool, ResetTool, BoxZoomTool


def plot_4_ele_(data):
    # 创建一个空列表来存储所有的图形对象
    plots = []

    # 循环遍历第四维度 (index from 0 to 3)
    for i in range(4):
        x_all = []
        y_all = []
        # 选择一个特定的维度进行绘图
        for n in range(len(data)):
            x_all.append(n)  # 序列号作为 x 轴
            y_all.append(data[n][i])  # 第四维度的值作为 y 轴

        # 创建一个图形对象
        p = figure(title=f"Dimension {i+1}", x_axis_label='Index (N)', y_axis_label=f'Dimension {i+1} Values',
                   tools=[PanTool(), WheelZoomTool(), BoxZoomTool(), ResetTool()])

        # 在图上绘制曲线
        p.line(x_all, y_all, line_width=2)

        # 保存该图形到列表中
        plots.append(p)

    # 创建一个 2x2 的布局，将所有图形显示在一个网格中
    grid = gridplot([plots[:2], plots[2:]])

    # 显示图形
    show(grid)

In [32]:
agent_enc_num = manager.agent_output_tensor().get_agent_enc_num_t()
agent_enc_num = agent_enc_num.data()
agent_enc_index = manager.agent_output_tensor().get_agent_enc_edge_index_t()
agent_enc_index = agent_enc_index.data()

agent_enc_index_list = []
for i in range(agent_enc_num[0]):
    agent_enc_index_list.append([agent_enc_index[0][i],agent_enc_index[1][i]])
agent_enc_num, agent_enc_index[0][:  10],agent_enc_index[1][:10]

([4785], [1, 1, 2, 1, 2, 3, 1, 2, 3, 4], [2, 3, 3, 4, 4, 4, 5, 5, 5, 5])

In [33]:
from bokeh.plotting import figure, show
from bokeh.models import GraphRenderer, StaticLayoutProvider, Circle, MultiLine
from bokeh.models.graphs import NodesAndLinkedEdges, EdgesAndLinkedNodes
import networkx as nx

# 示例数据
agent_enc_num = agent_enc_num[0] # 假设总共有6个索引
agent_enc_index = agent_enc_index_list

# 创建图形对象
G = nx.Graph()

# 添加节点
G.add_nodes_from(range(agent_enc_num))

# 添加边
for edge in agent_enc_index:
    G.add_edge(edge[0], edge[1])

# 使用 Bokeh 创建图形渲染器
graph = GraphRenderer()

# 节点属性
graph.node_renderer.data_source.add([str(i) for i in range(agent_enc_num)], 'index')
graph.node_renderer.glyph = Circle(radius=0.2, fill_color='skyblue')

# 边属性
start_indices = [edge[0] for edge in agent_enc_index]
end_indices = [edge[1] for edge in agent_enc_index]
graph.edge_renderer.data_source.data = dict(start=start_indices, end=end_indices)

# 创建静态布局
x = [i * 2 for i in range(agent_enc_num)]
y = [i % 3 * 2 for i in range(agent_enc_num)]
graph_layout = dict(zip(graph.node_renderer.data_source.data['index'], zip(x, y)))
graph.layout_provider = StaticLayoutProvider(graph_layout=graph_layout)

# 边和节点的互动效果
graph.node_renderer.selection_glyph = Circle(radius=0.3, fill_color='green')
graph.edge_renderer.selection_glyph = MultiLine(line_color='green', line_width=2)
graph.node_renderer.hover_glyph = Circle(radius=0.3, fill_color='orange')
graph.edge_renderer.hover_glyph = MultiLine(line_color='orange', line_width=2)
graph.selection_policy = NodesAndLinkedEdges()
graph.inspection_policy = EdgesAndLinkedNodes()

# 创建绘图
plot = figure(title="Agent Connectivity Graph", x_range=(-1, 120), y_range=(-1, 6),
              tools="", toolbar_location=None)

plot.renderers.append(graph)

# 显示图形
show(plot)


In [34]:
agent_enc_r_t = manager.agent_output_tensor().get_agent_enc_r_t()
agent_enc_r_t = agent_enc_r_t.data()
agent_enc_r_t[:10]

[[0.0, 0.0, 2.384185791015625e-07, -1.0],
 [0.0, 0.0, 4.76837158203125e-07, -2.0],
 [0.0, 0.0, 2.384185791015625e-07, -1.0],
 [0.0, 0.0, 7.152557373046875e-07, -3.0],
 [0.0, 0.0, 4.76837158203125e-07, -2.0],
 [0.0, 0.0, 2.384185791015625e-07, -1.0],
 [0.0, 0.0, 8.344650268554688e-07, -4.0],
 [0.0, 0.0, 5.960464477539062e-07, -3.0],
 [0.0, 0.0, 3.5762786865234375e-07, -2.0],
 [0.0, 0.0, 1.1920928955078125e-07, -1.0]]

In [35]:
plot_4_ele_(agent_enc_r_t)


In [36]:
agent_enc_num_a2a = manager.agent_output_tensor().get_agent_enc_num_a2a()
agent_enc_num_a2a = agent_enc_num_a2a.data()

agent_enc_edge_index_a2a = manager.agent_output_tensor().get_agent_enc_edge_index_a2a()
agent_enc_edge_index_a2a = agent_enc_edge_index_a2a.data()

get_agent_enc_r_a2a = manager.agent_output_tensor().get_agent_enc_r_a2a()
get_agent_enc_r_a2a = get_agent_enc_r_a2a.data()
agent_enc_num_a2a[0],agent_enc_edge_index_a2a[0][:10],agent_enc_edge_index_a2a[1][:10]

(2304,
 [15, 15, 15, 15, 15, 15, 15, 15, 15, 16],
 [16, 17, 18, 19, 20, 21, 22, 23, 24, 17])

In [37]:

# 创建一个空列表来存储所有的图形对象
plots = []

# 循环遍历第四维度 (index from 0 to 3)
for i in range(3):
    x_all = []
    y_all = []
    # 选择一个特定的维度进行绘图
    for n in range(len(get_agent_enc_r_a2a)):
        x_all.append(n)  # 序列号作为 x 轴
        y_all.append(get_agent_enc_r_a2a[n][i])  # 第四维度的值作为 y 轴

    # 创建一个图形对象
    p = figure(title=f"Dimension {i+1}", x_axis_label='Index (N)', y_axis_label=f'Dimension {i+1} Values')
    
    # 在图上绘制曲线
    p.line(x_all, y_all, line_width=2)
    
    # 保存该图形到列表中
    plots.append(p)

# 创建一个 2x2 的布局，将所有图形显示在一个网格中
grid = gridplot([plots[:2], plots[2:]])

# 显示图形
show(grid)

In [38]:
dec_num_t2m = manager.agent_output_tensor().get_dec_num_t2m()
dec_num_t2m = dec_num_t2m.data()

dec_edge_index_t2m = manager.agent_output_tensor().get_dec_edge_index_t2m()
dec_edge_index_t2m = dec_edge_index_t2m.data()

dec_r_t2m = manager.agent_output_tensor().get_dec_r_t2m()
dec_r_t2m = dec_r_t2m.data()
dec_num_t2m[0],dec_edge_index_t2m[0][:10],dec_r_t2m[:10]


(330,
 [20, 21, 22, 23, 24, 25, 26, 27, 28, 29],
 [[0.0001220703125, 1.789927363395691, 5.245208740234375e-06, -29.0],
  [0.0001220703125, 1.789927363395691, 4.887580871582031e-06, -28.0],
  [6.103515625e-05, 1.789927363395691, 4.649162292480469e-06, -27.0],
  [6.103515625e-05, 1.789927363395691, 4.5299530029296875e-06, -26.0],
  [6.103515625e-05, 1.789927363395691, 4.410743713378906e-06, -25.0],
  [6.103515625e-05, 1.789927363395691, 4.291534423828125e-06, -24.0],
  [6.103515625e-05, 1.789927363395691, 4.291534423828125e-06, -23.0],
  [6.103515625e-05, 1.789927363395691, 4.291534423828125e-06, -22.0],
  [6.103515625e-05, 1.789927363395691, 4.291534423828125e-06, -21.0],
  [6.103515625e-05, 1.789927363395691, 4.172325134277344e-06, -20.0]])

In [39]:
plot_4_ele_(dec_r_t2m)


In [40]:
import torch

In [41]:
a = torch.tensor([
    [1,0,1],
    [1,0,0],
    [1,1,1]
])
b = a.unsqueeze(2) & a.unsqueeze(1) 

In [42]:
from torch_geometric.utils import dense_to_sparse
edge_index_t = dense_to_sparse(b)[0]

In [43]:
edge_index_t

tensor([[0, 0, 2, 2, 3, 6, 6, 6, 7, 7, 7, 8, 8, 8],
        [0, 2, 0, 2, 3, 6, 7, 8, 6, 7, 8, 6, 7, 8]])

In [44]:
edge_index_t = edge_index_t[:, edge_index_t[1] > edge_index_t[0]]
edge_index_t = edge_index_t[:, edge_index_t[1] - edge_index_t[0] <= 1]
edge_index_t

tensor([[6, 7],
        [7, 8]])

In [45]:
torch.arange(5).repeat_interleave(2)

tensor([0, 0, 1, 1, 2, 2, 3, 3, 4, 4])

In [46]:
mask = torch.tensor([
    [1,0,0,1,1],
    [0,0,1,1,1]
])
mask_s = mask.transpose(0, 1).reshape(-1)
mask_s

tensor([1, 0, 0, 0, 0, 1, 1, 1, 1, 1])

In [47]:
from torch_geometric.data import HeteroData

data = HeteroData()
data['user'].x = torch.randn((5, 32))  # 5 个用户节点，每个节点 32 维特征
data['item'].x = torch.randn((10, 32))  # 10 个商品节点，每个节点 32 维特征

# 添加边
# data['user', 'interaction', 'item'].edge_index = torch.tensor([[0, 1, 2], [0, 1, 2]])

# 通过 DataLoader 进行批处理
from torch_geometric.loader import DataLoader

loader = DataLoader([data, data], batch_size=2)

for batch in loader:
    print('---',batch['user'].batch)
    print('***',batch['item'].batch)
    


--- tensor([0, 0, 0, 0, 0, 1, 1, 1, 1, 1])
*** tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])


In [48]:
loader = DataLoader([data, data,data, data, data,data], batch_size=2)

for batch in loader:
    print('---',batch['user'].batch)
    print('***',batch['item'].batch)

--- tensor([0, 0, 0, 0, 0, 1, 1, 1, 1, 1])
*** tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])
--- tensor([0, 0, 0, 0, 0, 1, 1, 1, 1, 1])
*** tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])
--- tensor([0, 0, 0, 0, 0, 1, 1, 1, 1, 1])
*** tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])


In [49]:
loader = DataLoader([data, data,data, data, data,data], batch_size=3)

for batch in loader:
    print('--',batch['user'].x.shape)
    print('---',batch['user'].batch)
    print('***',batch['item'].batch)
    print(batch.num_graphs)

-- torch.Size([15, 32])
--- tensor([0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2])
*** tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2])
3
-- torch.Size([15, 32])
--- tensor([0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2])
*** tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2])
3


In [50]:
pos_a = torch.tensor([
    [[1,1],[2,2],[3,3]],
    [[4,4],[5,5],[6,6]]
])
pos_m = pos_a

pos_s = pos_a.transpose(0, 1).reshape(-1, 2)
pos_s,pos_m.shape

(tensor([[1, 1],
         [4, 4],
         [2, 2],
         [5, 5],
         [3, 3],
         [6, 6]]),
 torch.Size([2, 3, 2]))

In [51]:
from torch_cluster import radius_graph
from torch_cluster import radius

In [52]:
pos_pl = torch.tensor([
    [7,7],[8,8],[9,9],[10,10]
])
pos_pl = pos_pl.repeat(3, 1)
pos_s,pos_pl

(tensor([[1, 1],
         [4, 4],
         [2, 2],
         [5, 5],
         [3, 3],
         [6, 6]]),
 tensor([[ 7,  7],
         [ 8,  8],
         [ 9,  9],
         [10, 10],
         [ 7,  7],
         [ 8,  8],
         [ 9,  9],
         [10, 10],
         [ 7,  7],
         [ 8,  8],
         [ 9,  9],
         [10, 10]]))

In [53]:
batch_s = torch.arange(3).repeat_interleave(2)
batch_pl = torch.arange(3).repeat_interleave(4)
batch_s,batch_pl

(tensor([0, 0, 1, 1, 2, 2]), tensor([0, 0, 0, 0, 1, 1, 1, 1, 2, 2, 2, 2]))

In [54]:
edge_index_pl2a = radius(x=pos_s[:, :2], y=pos_pl[:, :2], r=500.0, batch_x=batch_s, batch_y=batch_pl)
pos_pl,edge_index_pl2a

(tensor([[ 7,  7],
         [ 8,  8],
         [ 9,  9],
         [10, 10],
         [ 7,  7],
         [ 8,  8],
         [ 9,  9],
         [10, 10],
         [ 7,  7],
         [ 8,  8],
         [ 9,  9],
         [10, 10]]),
 tensor([[ 0,  0,  1,  1,  2,  2,  3,  3,  4,  4,  5,  5,  6,  6,  7,  7,  8,  8,
           9,  9, 10, 10, 11, 11],
         [ 0,  1,  0,  1,  0,  1,  0,  1,  2,  3,  2,  3,  2,  3,  2,  3,  4,  5,
           4,  5,  4,  5,  4,  5]]))

In [55]:
pos_m = pos_a[:, 2, :2]
pos_pl = torch.tensor([
    [7,7],[8,8],[9,9],[10,10]
])
pos_m

tensor([[3, 3],
        [6, 6]])

In [56]:
edge_index_pl2m = radius(
            x=pos_m[:, :2],
            y=pos_pl[:, :2],
            r=50.0,
            batch_x= None,
            batch_y= None,
            max_num_neighbors=300)
edge_index_pl2m

tensor([[0, 0, 1, 1, 2, 2, 3, 3],
        [0, 1, 0, 1, 0, 1, 0, 1]])

In [57]:
edge_index_pl2m = torch.cat([edge_index_pl2m + i * edge_index_pl2m.new_tensor(
            [[4], [2]]) for i in range(2)], dim=1)
edge_index_pl2m

tensor([[0, 0, 1, 1, 2, 2, 3, 3, 4, 4, 5, 5, 6, 6, 7, 7],
        [0, 1, 0, 1, 0, 1, 0, 1, 2, 3, 2, 3, 2, 3, 2, 3]])

In [58]:

a2m_radius = 2.0
edge_index_a2m = radius_graph(
            x=pos_m[:, :2],
            r=a2m_radius,
            batch= None,
            loop=False,
            max_num_neighbors=300)
edge_index_a2m

tensor([], size=(2, 0), dtype=torch.int64)

In [59]:
def bipartite_dense_to_sparse(adj: torch.Tensor) -> torch.Tensor:
    index = adj.nonzero(as_tuple=True)
    if len(index) == 3:
        batch_src = index[0] * adj.size(1)
        batch_dst = index[0] * adj.size(2)
        index = (batch_src + index[1], batch_dst + index[2])
    return torch.stack(index, dim=0)


In [60]:
# 5 3
valid_mask = torch.tensor([
    [0,1,1,1,1,1,1,1],
    [0,0,1,1,1,1,1,1],
    [0,1,1,1,1,1,0,0],
    [0,0,0,0,0,0,0,0]
])
pred_mask = torch.tensor([
    [0,0,0,0,0,1,1,1],
    [0,0,0,0,0,1,1,1],
    [0,0,0,0,0,1,0,0],
    [0,0,0,0,0,0,0,0]
])
pos_m = torch.tensor(
    [[1, 1],
     [2, 2],
     [3, 3],
     [0, 0]
    ])
num_historical_steps = 5

num_t2m_steps = 3
num_modes = 2
mask_src = valid_mask[:, :num_historical_steps].contiguous()
mask_src[:, :num_historical_steps - num_t2m_steps] = False
mask_dst = pred_mask.any(dim=-1, keepdim=True).repeat(1, num_modes)
mask_src.bool() , mask_dst, mask_dst[:, -1:], pos_m

(tensor([[False, False,  True,  True,  True],
         [False, False,  True,  True,  True],
         [False, False,  True,  True,  True],
         [False, False, False, False, False]]),
 tensor([[ True,  True],
         [ True,  True],
         [ True,  True],
         [False, False]]),
 tensor([[ True],
         [ True],
         [ True],
         [False]]),
 tensor([[1, 1],
         [2, 2],
         [3, 3],
         [0, 0]]))

In [61]:
mask_src = mask_src.bool()
mask_dst.unsqueeze(2) & mask_dst.unsqueeze(1)

tensor([[[ True,  True],
         [ True,  True]],

        [[ True,  True],
         [ True,  True]],

        [[ True,  True],
         [ True,  True]],

        [[False, False],
         [False, False]]])

In [62]:
edge_index_m2m = dense_to_sparse(
            mask_dst.unsqueeze(2) & mask_dst.unsqueeze(1))[0]
edge_index_m2m

tensor([[0, 0, 1, 1, 2, 2, 3, 3, 4, 4, 5, 5],
        [0, 1, 0, 1, 2, 3, 2, 3, 4, 5, 4, 5]])

In [63]:
edge_index_t2m = bipartite_dense_to_sparse(
            mask_src.unsqueeze(2) & mask_dst.unsqueeze(1))
edge_index_t2m

tensor([[ 2,  2,  3,  3,  4,  4,  7,  7,  8,  8,  9,  9, 12, 12, 13, 13, 14, 14],
        [ 0,  1,  0,  1,  0,  1,  2,  3,  2,  3,  2,  3,  4,  5,  4,  5,  4,  5]])

In [64]:

edge_index_a2m = radius_graph(
            x=pos_m[:, :2],
            r=2.0,
            batch= None,
            loop=False,
            max_num_neighbors=300)
print(edge_index_a2m)
edge_index_a2m = edge_index_a2m[:, mask_src[:, -1][edge_index_a2m[0]] & mask_dst[edge_index_a2m[1], 0]]
edge_index_a2m

tensor([[1, 3, 0, 2, 1, 0],
        [0, 0, 1, 1, 2, 3]])


tensor([[1, 0, 2, 1],
        [0, 1, 1, 2]])

In [65]:
edge_index_a2m = torch.cat(
            [edge_index_a2m + i * edge_index_a2m.new_tensor([3]) for i in
             range(2)], dim=1)
edge_index_a2m

tensor([[1, 0, 2, 1, 4, 3, 5, 4],
        [0, 1, 1, 2, 3, 4, 4, 5]])

In [66]:
dense = mask_src.unsqueeze(2) & mask_dst[:, -1:].unsqueeze(1)
index = dense.nonzero(as_tuple=True)
index

(tensor([0, 0, 0, 1, 1, 1, 2, 2, 2]),
 tensor([2, 3, 4, 2, 3, 4, 2, 3, 4]),
 tensor([0, 0, 0, 0, 0, 0, 0, 0, 0]))

In [67]:
edge_index_t2m = bipartite_dense_to_sparse(dense)
edge_index_t2m

tensor([[ 2,  3,  4,  7,  8,  9, 12, 13, 14],
        [ 0,  0,  0,  1,  1,  1,  2,  2,  2]])

In [68]:
mask_src.unsqueeze(2)

tensor([[[False],
         [False],
         [ True],
         [ True],
         [ True]],

        [[False],
         [False],
         [ True],
         [ True],
         [ True]],

        [[False],
         [False],
         [ True],
         [ True],
         [ True]],

        [[False],
         [False],
         [False],
         [False],
         [False]]])

In [69]:
dense_multi = mask_src.unsqueeze(2) & mask_dst.unsqueeze(1)
print(dense_multi)
dense_multi = bipartite_dense_to_sparse(dense_multi)
dense_multi

tensor([[[False, False],
         [False, False],
         [ True,  True],
         [ True,  True],
         [ True,  True]],

        [[False, False],
         [False, False],
         [ True,  True],
         [ True,  True],
         [ True,  True]],

        [[False, False],
         [False, False],
         [ True,  True],
         [ True,  True],
         [ True,  True]],

        [[False, False],
         [False, False],
         [False, False],
         [False, False],
         [False, False]]])


tensor([[ 2,  2,  3,  3,  4,  4,  7,  7,  8,  8,  9,  9, 12, 12, 13, 13, 14, 14],
        [ 0,  1,  0,  1,  0,  1,  2,  3,  2,  3,  2,  3,  4,  5,  4,  5,  4,  5]])

In [70]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GATConv
from torch_geometric.data import Data

# 图注意力网络模型
class NodeSizeGAT(nn.Module):
    def __init__(self, in_channels, out_channels, heads=1):
        super(NodeSizeGAT, self).__init__()
        # 使用图注意力网络
        self.gat_conv = GATConv(in_channels, out_channels, heads=heads, concat=False)
    
    def forward(self, x, edge_index):
        # 应用GATConv，获取新的节点表示
        x = self.gat_conv(x, edge_index)
        # 使用softmax将注意力权重标准化
        x = F.softmax(x, dim=1)
        return x

# 创建一个简单的图数据
node_features = torch.tensor([[1.0], [2.0], [3.0], [4.0], [5.0]], dtype=torch.float)
edge_index = torch.tensor([[0, 1, 2, 3],
                           [1, 2, 3, 4]], dtype=torch.long)

# 初始化模型
model = NodeSizeGAT(in_channels=1, out_channels=1)

# 构建数据
data = Data(x=node_features, edge_index=edge_index)

# 前向传播
output = model(data.x, data.edge_index)

print("Output Node Sizes:", output.detach().numpy())


Output Node Sizes: [[1.]
 [1.]
 [1.]
 [1.]
 [1.]]


In [71]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GATConv
from torch_geometric.data import Data, DataLoader
from torch_geometric.utils import to_networkx
import networkx as nx
# import matplotlib.pyplot as plt

# 定义模型
class NodeSizeGAT(nn.Module):
    def __init__(self, in_channels, out_channels, heads=1):
        super(NodeSizeGAT, self).__init__()
        self.gat_conv = GATConv(in_channels, out_channels, heads=heads, concat=False)
    
    def forward(self, x, edge_index):
        x = self.gat_conv(x, edge_index)
        return x

# 生成随机图数据
def generate_data(num_nodes, num_features):
    # 创建随机节点特征
    node_features = torch.randn(num_nodes, num_features)
    
    # 创建随机边
    edge_index = torch.randint(0, num_nodes, (2, num_nodes * 2), dtype=torch.long)
    
    # 计算目标节点大小（即输出）
    with torch.no_grad():
        target_size = torch.zeros(num_nodes, 1)
        for i in range(num_nodes):
            neighbors = edge_index[1][edge_index[0] == i]
            if len(neighbors) > 0:
                target_size[i] = 1 / neighbors.size(0) * node_features[neighbors].sum()
            else:
                target_size[i] = 0
    
    return Data(x=node_features, edge_index=edge_index, y=target_size)

# 生成数据集
num_graphs = 100
dataset = [generate_data(num_nodes=10, num_features=1) for _ in range(num_graphs)]

# 创建DataLoader
dataloader = DataLoader(dataset, batch_size=10, shuffle=True)

# 定义模型、损失函数和优化器
model = NodeSizeGAT(in_channels=1, out_channels=1)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# 训练模型
epochs = 200
for epoch in range(epochs):
    total_loss = 0
    for batch in dataloader:
        optimizer.zero_grad()
        out = model(batch.x, batch.edge_index)
        loss = criterion(out, batch.y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    
    if epoch % 20 == 0:
        print(f'Epoch {epoch+1}/{epochs}, Loss: {total_loss/len(dataloader):.4f}')

# 评估模型
test_data = generate_data(num_nodes=10, num_features=1)
model.eval()
with torch.no_grad():
    predicted = model(test_data.x, test_data.edge_index)
    print("Predicted Node Sizes:", predicted.squeeze().numpy())
    print("Actual Node Sizes:", test_data.y.squeeze().numpy())

# 可视化图
# G = to_networkx(test_data, to_undirected=True)
# plt.figure(figsize=(8, 8))
# pos = nx.spring_layout(G)
# nx.draw(G, pos, with_labels=True, node_color=predicted.squeeze().numpy(), cmap=plt.cm.Blues, node_size=500, font_color='white')
# plt.show()


/home/SENSETIME/fengxiaotong/miniconda3/envs/qcnet2/lib/python3.10/site-packages/torch_geometric/deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


Epoch 1/200, Loss: 0.5358
Epoch 21/200, Loss: 0.5174
Epoch 41/200, Loss: 0.5175
Epoch 61/200, Loss: 0.5167
Epoch 81/200, Loss: 0.5166
Epoch 101/200, Loss: 0.5166
Epoch 121/200, Loss: 0.5172
Epoch 141/200, Loss: 0.5162
Epoch 161/200, Loss: 0.5172
Epoch 181/200, Loss: 0.5161
Predicted Node Sizes: [-0.20215046 -0.2103501  -0.00748627  0.06517567 -0.12950625  0.0413271
 -0.07058493 -0.11644381 -0.01456868 -0.24192473]
Actual Node Sizes: [ 0.5629693  -1.160914    0.3369102   0.         -2.7664883   0.
 -0.07192722  0.12725696 -0.13736616  0.12643144]


Map tensor

In [72]:
num_points = torch.tensor([4,3,5,2])
num_polygons = 4
point_to_polygon_edge_index = torch.stack(
            [torch.arange(num_points.sum(), dtype=torch.long),
             torch.arange(num_polygons, dtype=torch.long).repeat_interleave(num_points)], dim=0)
point_to_polygon_edge_index

tensor([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13],
        [ 0,  0,  0,  0,  1,  1,  1,  2,  2,  2,  2,  2,  3,  3]])